In [3]:
import os
from PIL import Image
import numpy as np
from tqdm import tqdm

In [11]:
label_path = './dataset/Labeled_data/'
origin_path = './dataset/Original_data/'

label_list = os.listdir(label_path)
origin_list = os.listdir(origin_path)

print(label_list[0:5])
print(origin_list[0:5])

['0001TP_006690_L.png', '0001TP_006720_L.png', '0001TP_006750_L.png', '0001TP_006780_L.png', '0001TP_006810_L.png']
['0001TP_006690.png', '0001TP_006720.png', '0001TP_006750.png', '0001TP_006780.png', '0001TP_006810.png']


In [5]:
np.array(Image.open('./data_cap/Gray_data/train/0001TP_006690.png'))

array([[  3,   3,   3, ...,   3,   3,   3],
       [  3,   3,   3, ...,   3,   3,   3],
       [  3,   3,   3, ...,   3,   3,   3],
       ...,
       [  1,   1,   1, ..., 255, 255, 255],
       [  1,   1,   1, ..., 255, 255, 255],
       [  1,   1,   1, ..., 255, 255, 255]], dtype=uint8)

In [6]:
from collections import namedtuple


#--------------------------------------------------------------------------------
# Definitions
#--------------------------------------------------------------------------------

# a label and all meta information
Label = namedtuple( 'Label' , [

    'name'        , # The identifier of this label, e.g. 'car', 'person', ... .
                    # We use them to uniquely name a class

    'id'          , # An integer ID that is associated with this label.
                    # The IDs are used to represent the label in ground truth images
                    # An ID of -1 means that this label does not have an ID and thus
                    # is ignored when creating ground truth images (e.g. license plate).
                    # Do not modify these IDs, since exactly these IDs are expected by the
                    # evaluation server.

    'trainId'     , # Feel free to modify these IDs as suitable for your method. Then create
                    # ground truth images with train IDs, using the tools provided in the
                    # 'preparation' folder. However, make sure to validate or submit results
                    # to our evaluation server using the regular IDs above!
                    # For trainIds, multiple labels might have the same ID. Then, these labels
                    # are mapped to the same class in the ground truth images. For the inverse
                    # mapping, we use the label that is defined first in the list below.
                    # For example, mapping all void-type classes to the same ID in training,
                    # might make sense for some approaches.
                    # Max value is 255!

    'category'    , # The name of the category that this label belongs to

    'categoryId'  , # The ID of this category. Used to create ground truth images
                    # on category level.

    'hasInstances', # Whether this label distinguishes between single instances or not

    'ignoreInEval', # Whether pixels having this class as ground truth label are ignored
                    # during evaluations or not

    'color'       , # The color of this label
    ] )


#--------------------------------------------------------------------------------
# A list of all labels
#--------------------------------------------------------------------------------

# Please adapt the train IDs as appropriate for your approach.
# Note that you might want to ignore labels with ID 255 during training.
# Further note that the current train IDs are only a suggestion. You can use whatever you like.
# Make sure to provide your results using the original IDs and not the training IDs.
# Note that many IDs are ignored in evaluation and thus you never need to predict these!

labels = [
    #       name                     id    trainId   category            catId     hasInstances   ignoreInEval   color
    
    Label(  'Pole'                 ,  0 ,        0 , 'object'          , 0       , False        , True         , (  0,  0, 64) ),
    Label(  'Pole'                 ,  1 ,        0 , 'object'          , 0       , False        , True         , (192,192,128) ),
    Label(  'SignSymbol'           ,  2 ,        1 , 'object'          , 0       , False        , True         , (128,128, 64) ),
    Label(  'SignSymbol'           ,  3 ,        1 , 'object'          , 0       , False        , True         , (192,128,128) ),
    Label(  'SignSymbol'           ,  4 ,        1 , 'object'          , 0       , False        , True         , (  0, 64, 64) ),
    
    Label(  'Bicyclist'            ,  5 ,        2 , 'human'           , 1       , True         , False        , (  0,128,192) ),
    Label(  'Bicyclist'            ,  6 ,        2 , 'human'           , 1       , True         , False        , (192,  0,192) ),
    Label(  'Pedestrian'           ,  7 ,        3 , 'human'           , 1       , True         , False        , ( 64, 64,  0) ),
    Label(  'Pedestrian'           ,  8 ,        3 , 'human'           , 1       , True         , False        , (192,128, 64) ),
    Label(  'Pedestrian'           ,  9 ,        3 , 'human'           , 1       , True         , False        , ( 64,128, 64) ),
    Label(  'Pedestrian'           , 10 ,        3 , 'human'           , 1       , True         , False        , ( 64,  0,192) ),
    
    Label(  'Building'             , 11 ,        4 , 'construction'    , 2       , False        , False        , ( 64,  0, 64) ),
    Label(  'Building'             , 12 ,        4 , 'construction'    , 2       , False        , False        , (128,  0,  0) ),
    Label(  'Building'             , 13 ,        4 , 'construction'    , 2       , False        , False        , (192,  0,128) ),
    Label(  'Building'             , 14 ,        4 , 'construction'    , 2       , False        , False        , ( 64,192,  0) ),
    Label(  'Building'             , 15 ,        4 , 'construction'    , 2       , False        , False        , (  0,128, 64) ),
    Label(  'Fence'                , 16 ,        5 , 'construction'    , 2       , False        , False        , ( 64,128, 64) ),
    
    Label(  'Pavement'             , 17 ,        6 , 'flat'            , 3       , False        , False        , ( 64,192,128) ),
    Label(  'Pavement'             , 18 ,        6 , 'flat'            , 3       , False        , False        , (128,128,192) ),
    Label(  'Pavement'             , 19 ,        6 , 'flat'            , 3       , False        , False        , (  0,  0,192) ),
    Label(  'Road'                 , 20 ,        7 , 'flat'            , 3       , False        , False        , (192,  0, 64) ),
    
    Label(  'Car'                  , 21 ,        8 , 'vehicle'         , 4       , True         , False        , ( 64,128,192) ),
    Label(  'Car'                  , 22 ,        8 , 'vehicle'         , 4       , True         , False        , (128, 64, 64) ),
    Label(  'Car'                  , 23 ,        8 , 'vehicle'         , 4       , True         , False        , (192,128,192) ),
    Label(  'Car'                  , 24 ,        8 , 'vehicle'         , 4       , True         , False        , ( 64,  0,128) ),
    
    Label(  'Sky'                  , 25 ,        9 , 'sky'             , 5       , False        , False        , (128,128,128) ),
    
    Label(  'Tree'                 , 26 ,       10 , 'nature'          , 6       , False        , False        , (192,192,  0) ),
    Label(  'Tree'                 , 27 ,       10 , 'nature'          , 6       , False        , False        , (128,128,  0) ),
]

In [7]:
pixel2id = {' '.join(map(str, label.color)): label.trainId for label in labels}

In [8]:
new_label_path = './data_cap/gray_data2/'

In [13]:
for p in tqdm(label_list):
    img = np.array(Image.open(label_path + p))
    ret = np.array([[pixel2id[' '.join(map(str, pixel))] if ' '.join(map(str, pixel)) in pixel2id else 11 for pixel in row] for row in img], dtype=np.uint8)
    Image.fromarray(ret).save(new_label_path  + p)

100%|█████████████████████████████████████████████████████████████████████████████| 701/701 [37:19<00:00,  3.20s/it]


In [17]:
import shutil

train = []
f = open("./dataset/train.txt", 'r')
lines = f.readlines()
for line in lines:
    train.append(line.split('\n')[0]+'_L.png')
f.close()

val = []
f = open("./dataset/val.txt", 'r')
lines = f.readlines()
for line in lines:
    val.append(line.split('\n')[0]+'_L.png')
f.close()

test = []
f = open("./dataset/test.txt", 'r')
lines = f.readlines()
for line in lines:
    test.append(line.split('\n')[0]+'_L.png')
f.close()


for move_f in train:
    shutil.copy('./data_cap/gray_data2/'+move_f, './data_cap/gray_data2/train/'+move_f)
    
for move_f in val:
    shutil.copy('./data_cap/gray_data2/'+move_f, './data_cap/gray_data2/val/'+move_f)

for move_f in test:
    shutil.copy('./data_cap/gray_data2/'+move_f, './data_cap/gray_data2/test/'+move_f)